# EMG信号滤波器设计实验

本教程将教你如何设计和应用不同类型的滤波器来处理EMG信号。

## 学习内容
1. 带通滤波器（20-500 Hz）
2. 陷波滤波器（去除50/60 Hz工频干扰）
3. 低通和高通滤波器
4. 完整的预处理流程
5. 滤波前后效果对比

In [ ]:
# ===== 导入必要的Python库 =====

# 系统库
import sys  # 用于操作Python环境
from pathlib import Path  # 用于处理文件路径

# 科学计算库
import numpy as np  # 数值计算：数组、矩阵运算
import matplotlib.pyplot as plt  # 绘图库：创建图表
from scipy import signal as sp_signal  # 信号处理库：频谱分析、滤波
# 注意：这里重命名为sp_signal，避免与变量名signal冲突

# ===== 添加项目路径 =====
project_root = Path.cwd().parent  # 获取项目根目录
sys.path.insert(0, str(project_root))  # 添加到Python搜索路径

# ===== 导入自定义滤波器模块 =====
from code.week06_preprocessing.filters import EMGFilters
# 我们自己编写的EMG专用滤波器

# ===== 配置matplotlib =====
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']  # 中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.rcParams['figure.dpi'] = 100  # 图表清晰度

print("✓ 导入成功！")

## 1. 生成带噪声的EMG信号

创建一个包含多种噪声的EMG信号：
- 基础EMG信号
- 50Hz工频干扰
- 低频基线漂移
- 高频噪声

In [ ]:
# ===== 生成包含多种噪声的EMG信号 =====

# --- 第一步：设置基本参数 ---
fs = 1000  # 采样率：1000 Hz
duration = 5  # 时长：5秒
t = np.linspace(0, duration, int(duration * fs))  # 时间轴

# --- 第二步：生成纯净的EMG信号（60-150 Hz） ---
emg_clean = np.zeros(len(t))  # 初始化为全零数组
contraction_start = int(1 * fs)  # 收缩开始：1秒
contraction_end = int(4 * fs)    # 收缩结束：4秒

# 在收缩期添加多个频率成分（模拟肌肉激活）
for freq in range(70, 140, 15):  # 频率：70, 85, 100, 115, 130 Hz
    emg_clean[contraction_start:contraction_end] += 0.3 * np.sin(
        2 * np.pi * freq * t[contraction_start:contraction_end]
    )
# 这是我们想要保留的"纯净"EMG信号

# --- 第三步：添加50Hz工频干扰 ---
powerline_noise = 0.5 * np.sin(2 * np.pi * 50 * t)
# 工频干扰来自电网（中国/欧洲是50Hz，美国是60Hz）
# 振幅0.5 mV，是最常见的EMG噪声源
# 特点：固定频率，持续存在

# --- 第四步：添加低频基线漂移 ---
baseline_drift = 0.3 * np.sin(2 * np.pi * 0.5 * t)
# 基线漂移是缓慢的低频变化（0.5 Hz）
# 原因：电极接触不良、皮肤电阻变化、运动伪影
# 特点：频率很低，但幅度可能很大

# --- 第五步：添加高频噪声 ---
high_freq_noise = 0.1 * np.random.randn(len(t))
# 高频随机噪声（白噪声）
# 原因：电子设备噪声、环境电磁干扰
# 特点：随机、宽频、幅度较小

# --- 第六步：合成带噪声的信号 ---
emg_noisy = emg_clean + powerline_noise + baseline_drift + high_freq_noise
# 真实采集的EMG信号 = 纯净信号 + 各种噪声

# --- 第七步：打印各成分的统计信息 ---
print("✓ 信号生成完成")
print("\n各成分的统计特性（标准差）：")
print(f"  纯净EMG: {np.std(emg_clean):.4f} mV")
print(f"  工频噪声: {np.std(powerline_noise):.4f} mV")
print(f"  基线漂移: {np.std(baseline_drift):.4f} mV")
print(f"  高频噪声: {np.std(high_freq_noise):.4f} mV")
print(f"  合成信号: {np.std(emg_noisy):.4f} mV")
print("\n💡 标准差越大，信号/噪声的能量越大")

## 2. 可视化原始信号和噪声成分

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(14, 12))

# 纯净EMG
axes[0].plot(t, emg_clean, linewidth=0.8, color='green')
axes[0].set_ylabel('幅度 (mV)')
axes[0].set_title('1. 纯净EMG信号（60-150 Hz）', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 工频干扰
axes[1].plot(t, powerline_noise, linewidth=0.8, color='red')
axes[1].set_ylabel('幅度 (mV)')
axes[1].set_title('2. 50Hz工频干扰', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 基线漂移
axes[2].plot(t, baseline_drift, linewidth=2, color='orange')
axes[2].set_ylabel('幅度 (mV)')
axes[2].set_title('3. 低频基线漂移（0.5 Hz）', fontweight='bold')
axes[2].grid(True, alpha=0.3)

# 高频噪声
axes[3].plot(t[:1000], high_freq_noise[:1000], linewidth=0.5, color='gray')
axes[3].set_ylabel('幅度 (mV)')
axes[3].set_title('4. 高频随机噪声（仅显示前1秒）', fontweight='bold')
axes[3].grid(True, alpha=0.3)

# 合成的带噪声信号
axes[4].plot(t, emg_noisy, linewidth=0.5, color='blue')
axes[4].set_xlabel('时间 (秒)')
axes[4].set_ylabel('幅度 (mV)')
axes[4].set_title('5. 合成信号 = 1 + 2 + 3 + 4', fontweight='bold')
axes[4].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 应用带通滤波器（20-500 Hz）

带通滤波器保留EMG信号的有效频率范围，去除低频漂移和高频噪声。

In [ ]:
# ===== 应用带通滤波器（20-500 Hz） =====

# --- 第一步：创建滤波器对象 ---
filters = EMGFilters(fs=fs)  # 传入采样率，初始化滤波器

# --- 第二步：应用带通滤波器 ---
emg_bandpass = filters.bandpass_filter(
    emg_noisy,      # 输入：带噪声的信号
    lowcut=20,      # 低截止频率：20 Hz
    highcut=500     # 高截止频率：500 Hz
)
# 带通滤波器的作用：
# - 去除 < 20 Hz 的低频成分（基线漂移）
# - 去除 > 500 Hz 的高频成分（高频噪声）
# - 保留 20-500 Hz 的信号（EMG有效频率范围）

# --- 第三步：创建对比可视化（上下两图） ---
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 【上图】滤波前
axes[0].plot(t, emg_noisy, linewidth=0.5, color='blue', alpha=0.7, label='原始信号')
axes[0].set_ylabel('幅度 (mV)', fontsize=11)
axes[0].set_title('滤波前：包含所有噪声', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
# alpha=0.7: 透明度70%，让信号不那么刺眼

# 【下图】滤波后
axes[1].plot(t, emg_bandpass, linewidth=0.5, color='green', label='带通滤波后')
axes[1].set_xlabel('时间 (秒)', fontsize=11)
axes[1].set_ylabel('幅度 (mV)', fontsize=11)
axes[1].set_title('滤波后：保留20-500 Hz，去除了基线漂移', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# --- 第四步：定量评估滤波效果 ---
print("✓ 带通滤波完成")
print(f"\n滤波效果对比：")
print(f"  滤波前标准差: {np.std(emg_noisy):.4f} mV")
print(f"  滤波后标准差: {np.std(emg_bandpass):.4f} mV")
print(f"  标准差降低: {(1 - np.std(emg_bandpass)/np.std(emg_noisy))*100:.1f}%")

print("\n💡 观察要点：")
print("  - 基线漂移（缓慢的上下波动）已经消失")
print("  - 信号变得更平稳，围绕零点波动")
print("  - 但50Hz工频干扰仍然存在（需要陷波滤波器）")

## 4. 应用陷波滤波器（去除50Hz工频干扰）

陷波滤波器专门用于去除特定频率的干扰。

In [ ]:
# ===== 应用陷波滤波器去除50Hz工频干扰 =====

# --- 第一步：在带通滤波的基础上应用陷波滤波 ---
emg_notch = filters.notch_filter(
    emg_bandpass,   # 输入：带通滤波后的信号
    freq=50         # 要去除的频率：50 Hz
)
# 陷波滤波器（Notch Filter）：
# - 专门去除某个特定频率的干扰
# - 对其他频率影响很小
# - 类似在频谱上"挖一个坑"，所以叫"陷波"

# --- 第二步：创建三层对比图 ---
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# 【第一层】原始信号
axes[0].plot(t, emg_noisy, linewidth=0.5, color='blue', alpha=0.7)
axes[0].set_ylabel('幅度 (mV)')
axes[0].set_title('原始信号：包含50Hz干扰', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 【第二层】带通滤波后
axes[1].plot(t, emg_bandpass, linewidth=0.5, color='orange')
axes[1].set_ylabel('幅度 (mV)')
axes[1].set_title('带通滤波后：仍有50Hz干扰（看起来有规律的波纹）', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 【第三层】陷波滤波后
axes[2].plot(t, emg_notch, linewidth=0.5, color='green')
axes[2].set_xlabel('时间 (秒)')
axes[2].set_ylabel('幅度 (mV)')
axes[2].set_title('陷波滤波后：50Hz干扰已去除', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 陷波滤波完成")

print("\n💡 如何判断50Hz干扰存在？")
print("  - 时域：看到规律性的快速振荡（每秒50次）")
print("  - 频域：在50Hz处有明显的峰值（下个单元格会展示）")

print("\n💡 为什么需要两次滤波？")
print("  - 带通滤波：去除大范围的低频和高频噪声")
print("  - 陷波滤波：精确去除特定频率（50Hz）的干扰")
print("  - 两者结合，效果最佳")

## 5. 频域分析：观察滤波效果

In [ ]:
# 计算频谱
freqs_orig, psd_orig = sp_signal.welch(emg_noisy, fs=fs, nperseg=1024)
freqs_bp, psd_bp = sp_signal.welch(emg_bandpass, fs=fs, nperseg=1024)
freqs_notch, psd_notch = sp_signal.welch(emg_notch, fs=fs, nperseg=1024)

# 绘制频谱对比
plt.figure(figsize=(14, 6))

plt.semilogy(freqs_orig, psd_orig, linewidth=2, alpha=0.7, label='原始信号')
plt.semilogy(freqs_bp, psd_bp, linewidth=2, alpha=0.7, label='带通滤波后')
plt.semilogy(freqs_notch, psd_notch, linewidth=2, alpha=0.7, label='陷波滤波后')

# 标注关键频率
plt.axvline(20, color='green', linestyle='--', alpha=0.5, label='带通低截止')
plt.axvline(500, color='green', linestyle='--', alpha=0.5, label='带通高截止')
plt.axvline(50, color='red', linestyle='--', alpha=0.5, label='50Hz工频')

plt.xlabel('频率 (Hz)', fontsize=12)
plt.ylabel('功率谱密度', fontsize=12)
plt.title('滤波前后频谱对比', fontsize=14, fontweight='bold')
plt.xlim(0, 300)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("观察要点:")
print("  1. 原始信号在50Hz有明显峰值（工频干扰）")
print("  2. 带通滤波后低频成分减少，但50Hz峰值仍在")
print("  3. 陷波滤波后50Hz峰值消失")

## 6. 完整的预处理流程

使用EMGFilters的preprocess_emg方法，一次完成所有预处理。

In [ ]:
# 一键预处理
emg_processed = filters.preprocess_emg(
    emg_noisy,
    remove_powerline=True,
    powerline_freq=50
)

# 对比原始和处理后的信号
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 原始
axes[0].plot(t, emg_noisy, linewidth=0.5, color='red', alpha=0.7)
axes[0].set_ylabel('幅度 (mV)')
axes[0].set_title('原始EMG信号（未处理）', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-2, 2)

# 处理后
axes[1].plot(t, emg_processed, linewidth=0.5, color='green')
axes[1].set_xlabel('时间 (秒)')
axes[1].set_ylabel('幅度 (mV)')
axes[1].set_title('预处理后的EMG信号（带通+陷波+归一化）', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("✓ 完整预处理完成")
print(f"  原始信号 SNR: 无法计算")
print(f"  处理后信号更清晰，适合特征提取")

## 7. 不同滤波器参数的影响

实验不同的滤波器参数，观察对信号的影响。

In [ ]:
# 测试不同的带通滤波器参数
bp_configs = [
    (10, 500, '10-500 Hz（宽带）'),
    (20, 500, '20-500 Hz（标准）'),
    (50, 200, '50-200 Hz（窄带）')
]

fig, axes = plt.subplots(len(bp_configs) + 1, 1, figsize=(14, 12))

# 原始信号
axes[0].plot(t, emg_noisy, linewidth=0.5, color='gray', alpha=0.7)
axes[0].set_ylabel('幅度')
axes[0].set_title('原始信号', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 不同参数的滤波结果
for i, (lowcut, highcut, label) in enumerate(bp_configs, 1):
    filtered = filters.bandpass_filter(emg_noisy, lowcut=lowcut, highcut=highcut)
    axes[i].plot(t, filtered, linewidth=0.5)
    axes[i].set_ylabel('幅度')
    axes[i].set_title(f'带通滤波: {label}', fontweight='bold')
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('时间 (秒)')

plt.tight_layout()
plt.show()

print("观察不同参数的效果:")
print("  - 宽带（10-500 Hz）: 保留更多信息，但可能有更多噪声")
print("  - 标准（20-500 Hz）: 平衡的选择")
print("  - 窄带（50-200 Hz）: 信号最干净，但可能损失一些信息")

## 8. 实战：处理真实数据（可选）

In [ ]:
try:
    from code.week05_data_processing.data_loading import EMGDataLoader
    
    loader = EMGDataLoader(data_dir='../data/sample/', fs=1000)
    time_real, signals_real, label = loader.load_single_trial(
        '../data/sample/subject_01/fist/trial_001.csv'
    )
    
    # 处理第一个通道
    ch0_raw = signals_real[:, 0]
    ch0_filtered = filters.preprocess_emg(ch0_raw, remove_powerline=True)
    
    # 对比
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    axes[0].plot(time_real, ch0_raw, linewidth=0.5)
    axes[0].set_ylabel('幅度 (mV)')
    axes[0].set_title(f'真实数据 - 原始信号 ({label})', fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(time_real, ch0_filtered, linewidth=0.5, color='green')
    axes[1].set_xlabel('时间 (秒)')
    axes[1].set_ylabel('幅度 (mV)')
    axes[1].set_title('预处理后', fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✓ 真实数据处理完成")
    
except FileNotFoundError:
    print("⚠️  未找到样本数据")
    print("运行: python tools/generate_sample_data.py")

## 总结

### ✅ 学到的知识
1. **带通滤波器** - 保留EMG有效频率范围（20-500 Hz）
2. **陷波滤波器** - 去除特定频率干扰（50/60 Hz）
3. **完整预处理流程** - 一键完成所有滤波步骤
4. **参数选择** - 不同参数对信号的影响
5. **频域验证** - 使用频谱图验证滤波效果

### 🎯 关键要点
- EMG信号预处理是后续分析的基础
- 带通滤波器是最基本的预处理步骤
- 陷波滤波器专门去除工频干扰
- 滤波器参数需要根据应用场景调整
- 始终在频域验证滤波效果

### ⏭️ 下一步
- 学习 [03_特征提取实践.ipynb](03_特征提取实践.ipynb)
- 从预处理后的信号中提取有用特征
- 为机器学习分类做准备